# Weekly Aggregation of Weather Data

**Methodology:**
1. Extract Key Features from Weather Data and Trait Data files
2. Aggregate Weekly Weather Variables and Apply Padding for Missing Values past Harvest Date
3. Melt and Transpose Data



In [ ]:
import pandas as pd
import numpy as np
print(pd.__version__)

2.2.3


In [ ]:
weather_data_file = '4_Training_Weather_Data_2014_2023_full_year.csv'
df_weather = pd.read_csv(weather_data_file)
df_weather.head(10)

,Env,Date,RH2M,T2M_MAX,ALLSKY_SFC_SW_DWN,T2MWET,GWETTOP,QV2M,GWETPROF,T2M_MIN,T2MDEW,PS,T2M,GWETROOT,ALLSKY_SFC_PAR_TOT,WS2M,ALLSKY_SFC_SW_DNI,PRECTOTCORR
0,DEH1_2015,20150101,74.81,6.00,10.75,-1.61,0.77,2.81,0.74,-3.39,-3.86,102.08,0.63,0.77,50.83,3.13,26.76,0.00
1,DEH1_2015,20150102,79.81,8.23,5.72,0.59,0.77,3.42,0.74,-1.47,-1.08,102.44,2.26,0.77,28.82,2.02,2.59,0.00
2,DEH1_2015,20150103,92.81,7.77,1.74,3.99,0.78,4.94,0.74,-1.35,3.57,102.70,4.41,0.77,9.40,1.90,1.36,6.70
3,DEH1_2015,20150104,95.00,18.62,4.05,12.99,0.80,9.34,0.75,6.78,12.58,101.20,13.40,0.78,21.84,5.38,2.17,9.98
4,DEH1_2015,20150105,63.38,8.73,10.92,-0.87,0.79,2.87,0.76,-3.14,-4.08,102.41,2.34,0.79,51.96,5.17,27.70,0.00
5,DEH1_2015,20150106,79.69,1.90,4.37,-4.43,0.79,2.32,0.76,-5.10,-6.06,102.30,-2.81,0.78,21.81,1.98,4.13,0.95
6,DEH1_2015,20150107,64.25,-2.19,6.31,-7.35,0.78,1.71,0.76,-8.24,-10.27,102.01,-4.42,0.78,30.38,5.80,6.35,0.02
7,DEH1_2015,20150108,60.44,-5.10,11.16,-11.28,0.78,1.10,0.75,-11.54,-14.38,103.04,-8.20,0.78,51.29,4.57,28.26,0.00
8,DEH1_2015,20150109,65.31,2.86,8.73,-4.95,0.78,2.01,0.75,-5.38,-7.81,102.04,-2.10,0.78,42.17,4.63,15.98,0.01
9,DEH1_2015,20150110,65.25,-3.38,11.68,-9.45,0.78,1.34,0.75,-9.17,-12.27,103.25,-6.64,0.77,53.67,2.88,28.60,0.00


In [ ]:
# Select Weather Variables of Interest
df_weather_extracted = df_weather[["Env", "Date", "T2MDEW", "RH2M", "WS2M", "ALLSKY_SFC_SW_DWN", "T2M_MAX", "T2M_MIN", "PRECTOTCORR"]]

df_weather_extracted['Date'] = pd.to_datetime(df_weather_extracted['Date'], format='%Y%m%d')

In [ ]:
trait_data_file = '1_Training_Trait_Data_2014_2023.csv'
df_trait = pd.read_csv(trait_data_file)
df_trait.head(10)

,Env,Year,Field_Location,Experiment,Replicate,Block,Plot,Range,Pass,Hybrid,...,Stand_Count_plants,Pollen_DAP_days,Silk_DAP_days,Plant_Height_cm,Ear_Height_cm,Root_Lodging_plants,Stalk_Lodging_plants,Yield_Mg_ha,Grain_Moisture,Twt_kg_m3
0,DEH1_2014,2014,DEH1,G2F_2014_15,1,1,1,1.0,1.0,M0088/LH185,...,56.0,63.0,67.0,213.0,79.0,0.0,0.0,5.721725,20.8,706.664693
1,DEH1_2014,2014,DEH1,G2F_2014_15,1,1,2,1.0,2.0,M0143/LH185,...,54.0,61.0,63.0,286.0,172.0,0.0,0.0,11.338246,25.8,693.792841
2,DEH1_2014,2014,DEH1,G2F_2014_15,1,1,3,1.0,3.0,M0003/LH185,...,60.0,63.0,65.0,239.0,92.0,0.0,4.0,6.540810,20.8,698.941582
3,DEH1_2014,2014,DEH1,G2F_2014_15,1,1,4,1.0,4.0,M0035/LH185,...,59.0,61.0,63.0,242.0,118.0,0.0,0.0,10.366857,23.7,711.813434
4,DEH1_2014,2014,DEH1,G2F_2014_15,1,1,5,1.0,5.0,M0052/LH185,...,58.0,63.0,65.0,211.0,92.0,0.0,0.0,10.908814,19.4,743.993065
5,DEH1_2014,2014,DEH1,G2F_2014_15,1,1,6,1.0,6.0,M0008/LH185,...,59.0,63.0,63.0,244.0,124.0,0.0,0.0,11.510118,16.8,762.013659
6,DEH1_2014,2014,DEH1,G2F_2014_15,1,1,7,1.0,7.0,M0172/LH185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,DEH1_2014,2014,DEH1,G2F_2014_15,1,1,8,1.0,8.0,M0001/LH185,...,57.0,61.0,63.0,220.0,99.0,0.0,0.0,11.722743,19.7,724.685287
8,DEH1_2014,2014,DEH1,G2F_2014_15,1,1,9,1.0,9.0,M0014/LH185,...,56.0,61.0,61.0,216.0,103.0,0.0,0.0,1.354875,10.5,NaN
9,DEH1_2014,2014,DEH1,G2F_2014_15,1,1,10,1.0,10.0,M0352/LH185,...,48.0,61.0,61.0,216.0,106.0,0.0,2.0,7.894681,17.7,701.515953


In [ ]:
# Select Plant Date and Harvest Date associated with each environment
df_env_dates = df_trait[['Env', 'Date_Planted', 'Date_Harvested']]
df_env_dates_clean = df_env_dates.dropna()

df_env_dates_clean.loc[:, "Date_Planted"] = pd.to_datetime(df_env_dates_clean["Date_Planted"], format="%m/%d/%y").dt.date
df_env_dates_clean.loc[:, "Date_Harvested"] = pd.to_datetime(df_env_dates_clean["Date_Harvested"], format="%m/%d/%y").dt.date
df_unique_env_dates = df_env_dates_clean.drop_duplicates(subset=["Env", "Date_Planted", "Date_Harvested"])

In [ ]:
df_unique_env_dates.head(5)

,Env,Date_Planted,Date_Harvested
0,DEH1_2014,2014-05-05,2014-09-29
1,GAH1_2014,2014-04-04,2014-09-11
2,IAH1a_2014,2014-05-17,2014-10-21
3,IAH1b_2014,2014-05-17,2014-10-20
4,IAH1c_2014,2014-05-09,2014-10-20


In [ ]:
df_weather_extracted.head(5)

,Env,Date,T2MDEW,RH2M,WS2M,ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,PRECTOTCORR
0,DEH1_2015,2015-01-01,-3.86,74.81,3.13,10.75,6.00,-3.39,0.00
1,DEH1_2015,2015-01-02,-1.08,79.81,2.02,5.72,8.23,-1.47,0.00
2,DEH1_2015,2015-01-03,3.57,92.81,1.90,1.74,7.77,-1.35,6.70
3,DEH1_2015,2015-01-04,12.58,95.00,5.38,4.05,18.62,6.78,9.98
4,DEH1_2015,2015-01-05,-4.08,63.38,5.17,10.92,8.73,-3.14,0.00


#### Weekly aggregation, padding and wide to long formatting for 413 environments in weather data file.

In [ ]:
from datetime import timedelta

# Create an empty dictionary to store weather data for each unique (Env, Plant, Harvest) combo
weather_by_instance = {}

for _, row in df_unique_env_dates.iterrows():
    env = row['Env']
    plant_date = pd.to_datetime(row['Date_Planted'])
    harvest_date = pd.to_datetime(row['Date_Harvested'])
    start_date = plant_date - timedelta(weeks=10)   #start date is 10 week before Date_Planted

    # Filter the weather_data for the specific environment and date range
    filtered_weather = df_weather_extracted[
        (df_weather_extracted['Env'] == env) &
        (df_weather_extracted['Date'] >= start_date) &
        (df_weather_extracted['Date'] < harvest_date)
    ].copy()

    # Store filtered weather data  in the weather_by_instance dictionary with unique key
    key = (env, plant_date.strftime('%Y-%m-%d'), harvest_date.strftime('%Y-%m-%d'))
    weather_by_instance[key] = filtered_weather

In [ ]:
#add column for 'Week' (relative to Date_Planted) 
for key, df in weather_by_instance.items():
    env, plant_str, harvest_str = key
    plant_date = pd.to_datetime(plant_str)
    
    df['Week'] = ((df['Date'] - plant_date).dt.days // 7)
    

In [ ]:
#Weekly aggregation 
aggregated_weather_data = {}

for key, df in weather_by_instance.items():
    env, plant_str, harvest_str = key  # Extract env and dates
    plant_date = pd.to_datetime(plant_str)

    df_aggregated = df.groupby('Week').agg(
        T2MDEW=('T2MDEW', 'sum'),
        RH2M=('RH2M', 'mean'),
        WS2M=('WS2M', 'sum'),
        ALLSKY_SFC_SW_DWN=('ALLSKY_SFC_SW_DWN', 'sum'),
        T2M_MAX=('T2M_MAX', 'mean'),
        T2M_DIFF_SUM=('T2M_MAX', lambda x: (x - df.loc[x.index, 'T2M_MIN']).sum()),
        T2M_GDD=('T2M_MAX', lambda x: (((x + df.loc[x.index, 'T2M_MIN']) / 2) - 10).sum()),
        T2M_MIN=('T2M_MIN', 'mean'),
        PRECTOTCORR=('PRECTOTCORR', 'sum')
    ).reset_index()

    # Store the aggregated result in aggregated_weather_data dictionary
    aggregated_weather_data[key] = df_aggregated

#### Weekly Weather Features:
- **ALLSKY_SFC_SW_DWN** = Sum of solar radiation which reaches the Earth's surface 
- **WS2M** = Sum of Wind Speed
- **PRECTOTCORR** = Total rainfall
- **RH2M** = Mean Relative Humidity  
- **T2MDEW** = Sum of Dew/Frost Point  
- **T2M_DIFF_SUM** = Sum of Tmax - Tmin    
- **T2M_MAX** = Mean Tmax  
- **T2M_MIN** = Mean Tmin  
- **T2M_GDD** = Total heat which can contribute to growth <br>

- **GDD** = ∑((Tmax​+Tmin​)/2​−10)




In [ ]:
# Define the full range of weeks (-10 to 31)
full_weeks = np.arange(-10, 32)  

for key, df in aggregated_weather_data.items():
    
    # Create a dataframe with full range of weeks
    full_week_df = pd.DataFrame({'Week': full_weeks})
    
    # Merge with the df_aggregated in dictionary
    padded_df = full_week_df.merge(df, on='Week', how='left')
    
    # Fill missing values with 0
    padded_df.fillna(0, inplace=True)
    
    # Update df with padded values
    aggregated_weather_data[key] = padded_df

In [ ]:
# Example of padded values
aggregated_weather_data['DEH1_2014','2014-05-05','2014-09-29'].tail(12)

,Week,T2MDEW,RH2M,WS2M,ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_DIFF_SUM,T2M_GDD,T2M_MIN,PRECTOTCORR
30,20,94.95,74.427143,20.41,108.28,24.692857,78.67,63.515,13.454286,32.43
31,21,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
32,22,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
33,23,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
34,24,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
35,25,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
36,26,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
37,27,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
38,28,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00
39,29,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000,0.000000,0.00


In [ ]:
def transpose_data(filtered_weather_data):
    """
    Transpose df_aggregated and combine seperate df_aggregated into single df.

    For each (Environment, Date_Planted, Date_Harvested) group, this function reshapes 
    the weather data from wide to long format, then pivots it so each weather variable 
    per week becomes a separate column. The final output is a single DataFrame combining 
    all groups.

    Parameters:
        aggregated_weather_data (dictionary): 
            Dictionary with keys as (Environment, Date_Planted, Date_Harvested) and values 
            as DataFrames with aggregated weather data and corresponding week.

    Returns:
        pd.DataFrame: Combined DataFrame with transposed weather variables per group.
    """
    transposed_data_list = []

    for key, data in aggregated_weather_data.items():
        env, date_planted, date_harvested = key
        
        melted_data = data.melt(id_vars=['Week'], var_name='Variable', value_name='Value') #melt data from wide to long format
        melted_data['Env'] = env 
        melted_data['Date_Planted'] = date_planted 
        melted_data['Date_Harvested'] = date_harvested 
        melted_data['Week_Variable'] = melted_data['Variable'] + " - Week " + melted_data['Week'].astype(str) #create Week_variable combined col 

        transposed_data = melted_data.pivot(
            index=['Env', 'Date_Planted', 'Date_Harvested'], 
            columns='Week_Variable',  
            values='Value' 
        )  
        transposed_data.reset_index(inplace=True)
        transposed_data_list.append(transposed_data) 

    final_transposed_df = pd.concat(transposed_data_list, ignore_index=True) 

    return final_transposed_df

In [ ]:
df_transposed = transpose_data(aggregated_weather_data)

In [ ]:
df_transposed.head(5)

Week_Variable,Env,Date_Planted,Date_Harvested,ALLSKY_SFC_SW_DWN - Week -1,ALLSKY_SFC_SW_DWN - Week -10,ALLSKY_SFC_SW_DWN - Week -2,ALLSKY_SFC_SW_DWN - Week -3,ALLSKY_SFC_SW_DWN - Week -4,ALLSKY_SFC_SW_DWN - Week -5,ALLSKY_SFC_SW_DWN - Week -6,...,WS2M - Week 29,WS2M - Week 3,WS2M - Week 30,WS2M - Week 31,WS2M - Week 4,WS2M - Week 5,WS2M - Week 6,WS2M - Week 7,WS2M - Week 8,WS2M - Week 9
0,DEH1_2014,2014-05-05,2014-09-29,100.98,93.01,180.16,134.92,132.94,134.76,91.60,...,0.0,18.04,0.0,0.0,16.96,19.27,13.87,17.95,24.49,21.36
1,GAH1_2014,2014-04-04,2014-09-11,153.08,67.50,146.46,98.51,127.10,88.60,94.72,...,0.0,19.35,0.0,0.0,13.45,14.72,13.97,10.57,14.34,15.06
2,IAH1a_2014,2014-05-17,2014-10-21,109.78,114.57,154.81,88.07,120.98,130.39,140.41,...,0.0,18.02,0.0,0.0,25.20,14.26,22.45,18.51,14.86,20.32
3,IAH1b_2014,2014-05-17,2014-10-20,109.78,114.57,154.81,88.07,120.98,130.39,140.41,...,0.0,18.02,0.0,0.0,25.20,14.26,22.45,18.51,14.86,20.32
4,IAH1c_2014,2014-05-09,2014-10-20,155.14,86.00,92.67,120.51,128.77,126.89,101.45,...,0.0,20.56,0.0,0.0,17.41,25.55,12.43,23.22,18.31,15.53


In [ ]:
import re

metadata_cols = ['Env', 'Date_Planted', 'Date_Harvested']

week_cols = [col for col in df_transposed.columns if col not in metadata_cols]

def extract_week_number(col_name):
    match = re.search(r'Week (-?\d+)', col_name)
    return int(match.group(1)) if match else float('inf')

sorted_week_cols = sorted(week_cols, key=extract_week_number)

df_transposed = df_transposed[metadata_cols + sorted_week_cols]

In [ ]:
df_transposed.head(5)

Week_Variable,Env,Date_Planted,Date_Harvested,ALLSKY_SFC_SW_DWN - Week -10,PRECTOTCORR - Week -10,RH2M - Week -10,T2MDEW - Week -10,T2M_DIFF_SUM - Week -10,T2M_GDD - Week -10,T2M_MAX - Week -10,...,WS2M - Week 30,ALLSKY_SFC_SW_DWN - Week 31,PRECTOTCORR - Week 31,RH2M - Week 31,T2MDEW - Week 31,T2M_DIFF_SUM - Week 31,T2M_GDD - Week 31,T2M_MAX - Week 31,T2M_MIN - Week 31,WS2M - Week 31
0,DEH1_2014,2014-05-05,2014-09-29,93.01,4.55,76.304286,-34.53,66.62,-74.950,4.051429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GAH1_2014,2014-04-04,2014-09-11,67.50,18.81,80.685714,-5.32,73.49,-51.765,7.854286,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,IAH1a_2014,2014-05-17,2014-10-21,114.57,0.12,69.045714,-17.18,99.22,-43.620,10.855714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IAH1b_2014,2014-05-17,2014-10-20,114.57,0.12,69.045714,-17.18,99.22,-43.620,10.855714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,IAH1c_2014,2014-05-09,2014-10-20,86.00,6.79,81.088571,-95.50,86.73,-146.445,-4.725714,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_transposed['Growth_Cycle_Weeks'] = (
    (df_transposed['Date_Harvested'] - df_transposed['Date_Planted'])
    .dt.days / 7
).round()

df_transposed.drop(columns=['Date_Planted', 'Date_Harvested'], inplace=True)